# Fetching, Cleaning, and Converting SQL Results to DataFrames

## Introduction

This tutorial bridges two ways of working with SQLite in Python:

1. **Cursor-based querying** (`cursor.execute()` + `fetchall()`), which is useful for low-level control.
2. **Pandas-based querying** (`pd.read_sql()`), which is efficient for analysis.

To keep this notebook fully self-contained, you will **create a fresh SQLite database file** and load a small sample dataset first.

Run the cells top to bottom. You do not need to type anything yourself.

## Setup: Imports

In [ ]:
import sqlite3
import pandas as pd

## Create a Tutorial Database

This notebook creates a new SQLite database file named `coffee_shop_demo.db` in your current working directory.

- If the file already exists, this notebook will **overwrite** the tables used in this tutorial.
- This keeps the lesson repeatable and consistent.

In [ ]:
DB_PATH = 'coffee_shop_demo.db'
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()
print('Connected to:', DB_PATH)

## Create Tables and Insert Sample Data

We’ll create two tables:

- `customers`
- `orders`

Then we’ll insert a small amount of sample data so you can practice fetching and converting results.

In [ ]:
cursor.execute('DROP TABLE IF EXISTS orders;')
cursor.execute('DROP TABLE IF EXISTS customers;')

cursor.execute('''
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name  TEXT NOT NULL,
        last_name   TEXT NOT NULL,
        email       TEXT UNIQUE,
        city        TEXT,
        state       TEXT
    );
''')

cursor.execute('''
    CREATE TABLE orders (
        order_id    INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER NOT NULL,
        order_date  TEXT NOT NULL,
        total_cents INTEGER NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
''')

customers = [
    ('Ana', 'Lee', 'ana.lee@example.com', 'Louisville', 'KY'),
    ('Chris', 'Nguyen', 'chris.nguyen@example.com', 'Lexington', 'KY'),
    ('Maria', 'Soto', 'maria.soto@example.com', None, 'TN'),
    ('Derek', 'Wells', 'derek.wells@example.com', 'Cincinnati', None),
]

cursor.executemany('''
    INSERT INTO customers (first_name, last_name, email, city, state)
    VALUES (?, ?, ?, ?, ?);
''', customers)

# Map emails to IDs so orders reference real customers
cursor.execute('SELECT customer_id, email FROM customers;')
id_by_email = {email: cid for cid, email in cursor.fetchall()}

orders = [
    (id_by_email['ana.lee@example.com'], '2025-01-05', 2595),
    (id_by_email['ana.lee@example.com'], '2025-02-10', 1450),
    (id_by_email['maria.soto@example.com'], '2025-02-12',  895),
    (id_by_email['chris.nguyen@example.com'], '2025-03-01', 3195),
]

cursor.executemany('''
    INSERT INTO orders (customer_id, order_date, total_cents)
    VALUES (?, ?, ?);
''', orders)

connection.commit()
print('Tables created and sample data inserted.')

## What Does `fetchall()` Actually Return?

When you query with a cursor, SQLite returns results as Python data structures:

- `fetchall()` returns a **list**
- each row is a **tuple**
- column names are **not included** in the returned rows

In [ ]:
cursor.execute('''
    SELECT customer_id, first_name, last_name, city, state
    FROM customers
    ORDER BY last_name, first_name;
''')
rows = cursor.fetchall()
rows

In [ ]:
type(rows), type(rows[0])

## Extracting Column Names from the Cursor

After running a query, the cursor stores metadata about the result set in `cursor.description`.

We can use that to capture column names in the correct order.

In [ ]:
column_names = [desc[0] for desc in cursor.description]
column_names

## Converting Cursor Results to a DataFrame (Manual)

Now that we have:

- `rows` (the data)
- `column_names` (the headers)

We can build a DataFrame manually.

In [ ]:
df_customers = pd.DataFrame(rows, columns=column_names)
df_customers

## Basic Cleaning After SQL Queries

Even when your SQL is correct, results often need light cleanup before analysis.

Common examples:
- Fill missing values
- Rename columns
- Convert types (dates, currency)

Let’s do two quick examples below.

In [ ]:
# Check missing values
df_customers.isna().sum()

In [ ]:
# Fill missing values for analysis
df_customers_clean = df_customers.fillna('Unknown')
df_customers_clean

In [ ]:
# Rename columns to match a simpler analysis style
df_customers_clean = df_customers_clean.rename(columns={'first_name': 'first', 'last_name': 'last'})
df_customers_clean

## Example: Orders Table + Type Conversion

Next we’ll fetch `orders` and convert:

- `order_date` → datetime
- `total_cents` → dollars

In [ ]:
cursor.execute('''
    SELECT order_id, customer_id, order_date, total_cents
    FROM orders
    ORDER BY order_date;
''')
order_rows = cursor.fetchall()
order_columns = [desc[0] for desc in cursor.description]
df_orders = pd.DataFrame(order_rows, columns=order_columns)
df_orders

In [ ]:
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'])
df_orders['total_dollars'] = df_orders['total_cents'] / 100
df_orders

## Introducing `pd.read_sql()`

Now that you’ve seen the manual process, here’s the Pandas shortcut.

`pd.read_sql()`:
- sends the SQL query
- fetches results
- preserves column names
- returns a DataFrame immediately

In [ ]:
df_customers_sql = pd.read_sql('''
    SELECT customer_id, first_name, last_name, city, state
    FROM customers
    ORDER BY last_name, first_name;
''', connection)

df_customers_sql

## When to Use Each Approach

### Cursor-based approach
Use this when you need:
- `INSERT`, `UPDATE`, `DELETE`
- transactions (`commit` / `rollback`)
- parameterized queries
- detailed debugging and control

### Pandas-based approach
Use this when you want:
- analysis-ready DataFrames
- fast exploration (`pd.read_sql`)
- quick inspection of tables and joins
- you are building the database from existing Pandas DataFrames

In real projects, you often use **both**.

## Cleanup

Always close the connection when you’re done.

In [ ]:
connection.close()
print('Connection closed.')